# Fetch and Clean Journey Bike Data

In [2]:
import pandas as pd
import requests
import io
import json
import urllib
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import datetime
from rapidfuzz import fuzz
from concurrent import futures

## 1. IMPORT BIKE JOURNEY DATA

### Fetch data
Due to the dynamic loading of the data, web scraping is not possible. Therefore, the name of the files are copy & pased into a CSV file which then get fetched and combined with multiple API requests.

In [ ]:
def rename_columns(df):
    """
    This function renames the columns in the provided dataframe 'df' as per the 
    mapping defined in 'column_names', and also changes the datatype of some columns.
    """
    
    # define a mapping of old column names to new standardized names
    column_names = {
        'End Station Id': 'EndStation Id',
        'End station number': 'EndStation Id',
        'Start Station Id': 'StartStation Id',
        'Start station number': 'StartStation Id',
        'End Station Name': 'EndStation Name',
        'End station': 'EndStation Name',
        'Start Station Name': 'StartStation Name',
        'Start station': 'StartStation Name',
        'Start date': 'Start Date',
        'End Date': 'End Date',
        'End date': 'End Date',
        'Number': 'Rental Id',
    }
    
    for old_name, new_name in column_names.items():
            if old_name in df.columns:
                df = df.rename(columns={old_name: new_name})
                if new_name in ['EndStation Id', 'StartStation Id', 'Rental Id']:
                    df[new_name] = pd.to_numeric(df[new_name], errors='coerce', downcast='integer')
                elif new_name in ['Start Date', 'End Date']:
                    df[new_name] = pd.to_datetime(df[new_name], infer_datetime_format=True)
    
    return df

In [ ]:
# load the list of file names from a CSV file
filenames = pd.read_csv('/Users/tabea/Documents/UrbanMobility/filenames-data.csv', header=None, squeeze=True)

# combine a list of URL by add the base-url and filename
base_url = 'http://cycling.data.tfl.gov.uk/usage-stats/'
url_list = (base_url + urllib.parse.quote(x) for x in filenames)
unused_cols = ['Total duration (ms)', 'Total duration', 'Duration', 'Duration_Seconds', 'Bike Id', 'Bike number', 'Bike model']

# loop through each URL to extract data
temp_dfs = []
for url in url_list:
    response = requests.get(url, verify=False, timeout=(3, 7))

    if url.endswith('.csv'):
        temp_df = pd.read_csv(io.StringIO(response.content.decode('utf-8')), usecols=lambda col: col not in unused_cols)

    elif url.endswith('.xlsx'):
        temp_df = pd.read_excel(io.BytesIO(response.content), usecols=lambda col: col not in unused_cols)

    temp_df = rename_columns(temp_df)
    temp_dfs.append(temp_df)

# concatenate all temporary dataframes into a single dataframe
merged_df = pd.concat(temp_dfs, ignore_index=True)


In [12]:
# total amount of entries: 84'188'068
len(merged_df)

84188068

In [69]:
merged_df.to_csv('/Users/tabea/Documents/UrbanMobility/data/journey_data_raw.csv')
merged_df.head()

,Unnamed: 0,Unnamed: 0.1,Rental Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name,Unnamed: 9,Unnamed: 10,Unnamed: 11,EndStation Logical Terminal,endStationPriority_id,StartStation Logical Terminal
0,0,0,63097899.0,2017-03-15 00:06:00,631.0,"Battersea Park Road, Nine Elms",2017-03-15 00:00:00,74.0,"Vauxhall Cross, Vauxhall",NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,63097900.0,2017-03-15 00:05:00,397.0,"Devonshire Terrace, Bayswater",2017-03-15 00:01:00,410.0,"Edgware Road Station, Marylebone",NaN,NaN,NaN,NaN,NaN,NaN
2,2,2,63097901.0,2017-03-15 00:06:00,426.0,"Vincent Street, Pimlico",2017-03-15 00:01:00,177.0,"Ashley Place, Victoria",NaN,NaN,NaN,NaN,NaN,NaN
3,3,3,63097902.0,2017-03-15 00:12:00,462.0,"Bonny Street, Camden Town",2017-03-15 00:01:00,22.0,"Northington Street , Holborn",NaN,NaN,NaN,NaN,NaN,NaN
4,4,4,63097903.0,2017-03-15 00:05:00,423.0,"Eaton Square (South), Belgravia",2017-03-15 00:01:00,143.0,"Pont Street, Knightsbridge",NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
# import data from disk (if already fetched)
# merged_df = pd.read_csv('/Users/tabea/Documents/UrbanMobility/data/journey_data_raw.csv')

## 2. CLEAN BIKE JOURNEY DATA

In [70]:
print("length before cleaning:", len(merged_df))

length before cleaning: 84188068


### Drop columns starting with 'Unnamed'

In [71]:
merged_df = merged_df.filter(regex='^(?!Unnamed)')

### Drop rows with nan values only

In [72]:
merged_df = merged_df.dropna(how='all')

### Investigate and drop duplicates

In [73]:
# some files have same or overlapping content, but different names. 
# e.g: 01b Journey Data Extract 24Jan16-06Feb16.csv, 01bJourneyDataExtract24Jan16-06Feb16.csv
    
duplicates_rental_id = merged_df[merged_df['Rental Id'].duplicated(keep=False)]
print("duplicated rental ID samples count: ", len(duplicates_rental_id))
# duplicates_rental_id.to_csv('/Users/tabea/Documents/UrbanMobility/data/duplicates.csv')

duplicated rental ID samples count:  585398


In [74]:
# drop all samples with duplicated rental id, sort first to keep the row with the most non-null values

merged_df['nonnull_count'] = merged_df.notnull().sum(axis=1)
merged_df = merged_df.sort_values(by=['Rental Id', 'nonnull_count'], ascending=[True, False])
merged_df = merged_df.drop_duplicates(subset='Rental Id', keep='first')
merged_df = merged_df.drop(columns='nonnull_count')

print("current length of df: ", len(merged_df))

current length of df:  83895356


### Investigate all nan values

In [75]:
print(merged_df.isna().sum())

Rental Id                               0
End Date                           170358
EndStation Id                      715522
EndStation Name                    171824
Start Date                              0
StartStation Id                    234440
StartStation Name                       0
EndStation Logical Terminal      83665717
endStationPriority_id            83665717
StartStation Logical Terminal    83662856
dtype: int64


### NaN values: StartStation Name & EndStation Name

In [77]:
StartStationName_NAN = merged_df[merged_df["StartStation Name"].isna()]
print("StartStation Name NaNs count: ", len(StartStationName_NAN))

EndStationName_NAN = merged_df[merged_df["EndStation Name"].isna()]
print("EndStation Name NaNs: count  ", len(EndStationName_NAN))
# EndStationName_NAN.to_csv('/Users/tabea/Documents/UrbanMobility/data/EndStationName_NAN.csv')

StartStation Name NaNs count:  0
EndStation Name NaNs: count   171824


In [78]:
# EndStation Name is only NaN if EndStation ID is also NaN -> they can't be mapped, so they must be removed.

merged_df = merged_df.dropna(subset=['EndStation Id', 'EndStation Name'], how='all')
print("current df length:", len(merged_df))

current df length: 83723532


### NaN values: Start Date and End Date

In [80]:
StartDate_NAN = merged_df[merged_df["Start Date"].isna()]
print("Start Date NaNs: ", len(StartDate_NAN))

EndDate_NAN = merged_df[merged_df["End Date"].isna()]
print("End Date NaNs: ", len(EndDate_NAN))
# EndDate_NAN.to_csv('/Users/tabea/Documents/UrbanMobility/data/EndDate_NAN.csv')

Start Date NaNs:  0
End Date NaNs:  69


In [81]:
# drop 69 entries with missing data

merged_df = merged_df.dropna(subset=['End Date'])
print("current df length:", len(merged_df))

current df length: 83723463


### NaN values: StartStation Id & EndStation Id
Numerous NaN values are observed in the 'StartStation Id' and 'EndStation Id' columns. The primary cause: cycling rides extending beyond a single calendar day. For these instances, stations are referred to as 'TerminalStation', each carrying a unique ID set with higher numbers (>852).

Due to the mix of stationID and terminalID and lots of NaN values, the ID's get dropped and the name of the station is used as identifier.

In [82]:
# StartStation Id: 234'440 NaN -> but most StartStation Names are known
StartStationId_NAN = merged_df[merged_df["StartStation Id"].isna()]
print("StartStation Id NaNs count: ", len(StartStationId_NAN))
# StartStationId_NAN.to_csv('/Users/tabea/Documents/UrbanMobility/data/StartStationId_NAN.csv')

# EndStation Id: 715'522 NaN -> but most EndStation Names are known
EndStationId_NAN = merged_df[merged_df["EndStation Id"].isna()]
print("EndStation Id NaNs count: ", len(EndStationId_NAN))
# EndStationId_NAN.to_csv('/Users/tabea/Documents/UrbanMobility/data/EndStationId_NAN.csv')

StartStation Id NaNs count:  231579
EndStation Id NaNs count:  543698


In [83]:
# only 852 station are present in the data. But there are also terminal station IDs that have higher values and are mixed in the data.
# they can be found here: https://api.tfl.gov.uk/BikePoint/

greater_than_852 = (merged_df['StartStation Id'] > 852) | (merged_df['EndStation Id'] > 852)
print("count of terminal station ID instaed of normal ID: ", greater_than_852.sum())

count of terminal station ID instaed of normal ID:  2788522


In [84]:
# drop ID's

merged_df = merged_df.drop(columns=['StartStation Id', 'EndStation Id', 'EndStation Logical Terminal', 'endStationPriority_id', 'StartStation Logical Terminal'])
print(merged_df.isna().sum())

Rental Id            0
End Date             0
EndStation Name      0
Start Date           0
StartStation Name    0
dtype: int64


### Change dtypes

In [ ]:
merged_df["Rental Id"] = merged_df["Rental Id"].astype(int)
merged_df["Start Date"] = pd.to_datetime(merged_df["Start Date"])
merged_df["End Date"] = pd.to_datetime(merged_df["End Date"])

### Delete journeys from 2015 and 2023 (only start from April -> not complete)

In [ ]:
merged_df['End Date'] = pd.to_datetime(merged_df['End Date'])
mask_2015 = merged_df['End Date'].dt.year == 2015
merged_df = merged_df[~mask_2015]

merged_df['Start Date'] = pd.to_datetime(merged_df['Start Date'])
mask_2015 = merged_df['Start Date'].dt.year == 2015
merged_df = merged_df[~mask_2015]

merged_df['End Date'] = pd.to_datetime(merged_df['End Date'])
mask_2023 = merged_df['End Date'].dt.year == 2023
merged_df = merged_df[~mask_2023]

merged_df['Start Date'] = pd.to_datetime(merged_df['Start Date'])
mask_2023 = merged_df['Start Date'].dt.year == 2023
merged_df = merged_df[~mask_2023]

### Rename columns

In [ ]:
merged_df = merged_df.rename(columns={
    'Rental Id': 'rental_id',
    'End Date': 'end_date',
    'EndStation Name': 'endStation_name',
    'Start Date': 'start_date',
    'StartStation Name': 'startStation_name'
})

# 3. SAVE CLEANED DATA
464'605 samples got deleted

In [89]:
print("length df after cleaning:", len(merged_df))
# merged_df.to_csv('/Users/tabea/Documents/UrbanMobility/data/journey_data_cleaned.csv')

length df after cleaning: 83723463
